In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Inlezen

De eerste data set bevat de aanwezigheidsgegevens uit Canvas van studenten bij de lessen Programming gedurende een blok. Het `Student ID` hier is het identificatienummer van __Canvas__.

In [ ]:
df_prog = pd.read_csv('pandas_attendance_prog.csv', delimiter=';')

In [ ]:
df_prog.head()

In [ ]:
df_prog.drop(['Course ID', 'Teacher ID', 'Course Name', 'Timestamp'], axis=1, inplace=True)

In [ ]:
df_prog.columns

In [ ]:
df_prog.rename(columns={'Teacher Name': 'Docent'}, inplace=True)

In [ ]:
df_prog.head()

De tweede dataset bevat gegevens over de gebruikers van Canvas, waaronder een studentnummer in het geval van een student. (Veel gegevens, zoals achternaam en emailadres, zijn hier verwijderd vanwege privacy).

In [ ]:
df_users = pd.read_csv('pandas_attendance_users.csv', sep=';', encoding='utf-8')

In [ ]:
df_users.head()

## Lege waarden

In [ ]:
df_users.isnull().head()

In [ ]:
len(df_users)

In [ ]:
len(df_users.dropna())

In [ ]:
len(df_users.dropna(subset=['status']))

In [ ]:
df_users.dropna(inplace=True)

## Datatypes

In [ ]:
df_users.describe()

Wat voor datatype is `user_id` nu eigenlijk? En wat zou het moeten zijn?

In [ ]:
df_users['user_id'] = df_users['user_id'].astype(int)

In [ ]:
df_users['user_id'] = df_users['user_id'].astype('category')
df_users['canvas_id'] = df_users['canvas_id'].astype('category')

In [ ]:
df_users.set_index('user_id', inplace=True)

In [ ]:
df_users.head()

## Multilevel indexing

De laatste dataset bevat de inschrijvingen van een student in bepaalde vakken en de klas waarin hij of zij zit uit Osiris.

In [ ]:
df_students = pd.read_csv('pandas_attendance_students.csv', sep=';')

In [ ]:
df_students.head()

In [ ]:
len(df_students)

In [ ]:
df_students.dtypes

In [ ]:
df_students['user_id'] = df_students['user_id'].astype('category')
df_students.drop(['role', 'status'], axis=1, inplace=True)

In [ ]:
df_students['course_id'].unique()

In [ ]:
len(df_students['user_id'].unique())

In [ ]:
df_students.set_index(['course_id', 'section_id'], inplace=True)

In [ ]:
df_students.head()

In [ ]:
df_students.loc['TICT-V1PROG'].head()

In [ ]:
df_students.loc[ [('TICT-V1PROG', 'TICT-ICT-V1A'),('TICT-V1CSN', 'TICT-ICT-V1B')]]

In [ ]:
len(df_users)

In [ ]:
len(df_students)

## Mergen en groeperen

We gaan de gegevens van de Canvasgebruikers samenvoegen met de gegevens uit Osiris, zodat we weten welke Canvasgebruiker zich heeft ingeschreven voor welk vak en tot welke klas hij/zij behoort.

In [ ]:
df_students.reset_index(inplace=True)

In [ ]:
df_students.set_index(['user_id'], inplace=True)

In [ ]:
df_students.index.unique()

In [ ]:
len(df_students.index)

In [ ]:
df_users.index.unique()

In [ ]:
len(df_users.index)

Blijkbaar komt een studentnummer meerdere malen voor in `df_students` (dat klopt, elke student is namelijk aan een klas gekoppeld voor élk van de vakken PROG, CSN en ICOR) en maar één maal in `df_users`, maar in de laatste zijn wel weer méér unieke studentnummers te vinden (dat klopt ook, namelijk de nummers van álle studenten op Canvas, niet alleen van deze vakken).

In [ ]:
len(df_users.merge(df_students, how='left', left_index=True, right_index=True))

Een left join levert veel te veel combinaties op, we willen alleen de inschrijvingen voor PROG overhouden. Laten we een inner join proberen met een filter erbij.

In [ ]:
len(df_users.merge(df_students[df_students['course_id'] == 'TICT-V1PROG'], how='inner', left_index=True, right_index=True))

In [ ]:
df_users.merge(df_students[df_students['course_id'] == 'TICT-V1PROG'], how='inner', left_index=True, right_index=True).head()

In [ ]:
df_all_users = df_users.merge(df_students[df_students['course_id'] == 'TICT-V1PROG'], how='inner', left_index=True, right_index=True)

In [ ]:
df_all_users.head()

Index verwijderen, anders worden deze bij de volgende handmatige merge niet meegenomen.

In [ ]:
df_all_users.reset_index(inplace=True)

Als laatste gaan we (handmatig) mergen met de aanwezigheidsinformatie van PROG.

In [ ]:
df_prog['Student ID'] = df_prog['Student ID'].astype('category')

In [ ]:
df_attendance = df_prog.merge(df_all_users, how='inner', 
                              left_on=['Student ID', 'Course Code'], 
                              right_on=['canvas_id','course_id'])

In [ ]:
df_attendance['Student ID'] = df_attendance['Student ID'].astype('category')
df_attendance['user_id'] = df_attendance['user_id'].astype('category')

In [ ]:
df_attendance.size

In [ ]:
df_attendance['Attendance'].unique()

We vinden iemand aanwezig als deze _niet_ absent was.

In [ ]:
df_attendance['Aanwezig'] = (df_attendance['Attendance'] != 'absent').astype(int)

In [ ]:
df_attendance.head()

Laten we de aanwezigheid groeperen op klas.

In [ ]:
df_attendance_klas = df_attendance.groupby(['section_id']).sum()

In [ ]:
df_attendance_klas.head(5)

In [ ]:
fig, ax = plt.subplots(figsize=(12,4), dpi=150)
ax.bar(df_attendance_klas.index, df_attendance_klas['Aanwezig'])
plt.xticks(rotation=90)
plt.show()

## `datetime` data type

Laten we eens groeperen alléén op dag.

In [ ]:
df_attendance_daily = df_attendance.groupby(['Class Date']).sum()

In [ ]:
df_attendance_daily.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12,4), dpi=150)
ax.bar(df_attendance_daily.index, df_attendance_daily['Aanwezig'])
plt.xticks(rotation=90)
plt.show()

In [ ]:
df_attendance_daily.index

De x-as (dag) is blijkbaar nominaal geïnterpreteerd ('object'), terwijl tijd natuurlijk gewoon kwantitatief is: laten we het datatype wijzigen om dat te reflecteren.

In [ ]:
df_attendance_daily.index = pd.to_datetime(df_attendance_daily.index, format='%d-%m-%Y')

In [ ]:
df_attendance_daily.index

In [ ]:
fig, ax = plt.subplots(figsize=(12,4), dpi=150)
ax.bar(df_attendance_daily.index, df_attendance_daily['Aanwezig'])
plt.show()

Het `datetime` type kunnen we nu ook gaan resamplen, bijvoorbeeld naar 'per week'.

In [ ]:
df_attendance_weekly = df_attendance_daily.resample('W', convention='start').sum()

In [ ]:
df_attendance_weekly.index

In [ ]:
df_attendance_weekly.index.week

In [ ]:
fig, ax = plt.subplots(figsize=(12,4), dpi=150)
ax.bar(df_attendance_weekly.index.week, df_attendance_weekly['Aanwezig'])
plt.show()

## Oefening

Toon een overzicht van de totale aanwezigheid per student van klas TICT-ICT-V1E voor het vak PROG, gesorteerd op aanwezigheid.

## Oefening

Toon een overzicht van de totale aanwezigheid per weekdag (maandag, dinsdag, ...).